In [ ]:
import numpy as np
import scipy
import os
import glob
import matplotlib.pyplot as plt
from IPython.display import Image as disImage
from PIL import Image
import csv
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

In [ ]:
#display images pulling from os
directory = '/home/logan/Desktop/MATH/SampleMoviePosters/SampleMoviePosters'
for dirname, _, filenames in os.walk(directory):
    for file in filenames:
        os.path.join(dirname, file)
#disImage(filename = '/home/logan/Desktop/MATH/SampleMoviePosters/SampleMoviePosters/2461.jpg')

In [ ]:
#create list of movie id's
id_list = []
for filename in glob.iglob(f'{directory}/*'):
    f = filename.split('/')
    id_list.append(f[-1][:-4])
    #print(f[-1][:-4])

In [ ]:
#get posters as np arrays
shape = (268,182,3)
image_array = []
for each in id_list:
    img = Image.open('/home/logan/Desktop/MATH/SampleMoviePosters/SampleMoviePosters/'+each+'.jpg')
    image_array.append(np.array(img))
    #print (img)

In [ ]:
#create dictionaries
id_genre_dict = {}
title_dict = {}
with open('/home/logan/Downloads/MovieGenre.csv', newline='',encoding = "ISO-8859-1") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        movie_id = row[0]
        gen = row[4]
        title = row[2]
        gen_split = gen.split('|')
        id_genre_dict.update({movie_id:gen_split})
        title_dict.update({movie_id:title})
        #print(row)

In [ ]:
genres = id_genre_dict.values()
genres = [item for sublist in genres for item in sublist]
genres = set(genres)
genres = list(genres)
genres.remove('')
genres.remove('Genre')
genres.sort()
#print(genres)

In [ ]:
arr_y = []
for each in id_list:
    y_each = np.zeros(len(genres))
    gens = id_genre_dict[each]
    for i in gens:
        y_each[genres.index(i)] = 1
    arr_y.append(y_each)
#id_genre_dict[id_list[0]], arr_y[0]

In [ ]:
img = np.asarray(image_array)
np.save('X.npy',img)
y = np.asarray(arr_y)
np.save('Y',y)
#print(img)
#print(Y)

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size = 3, activation = 'relu', input_shape = (268,182,3)))
model.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
model.add(Flatten())
model.add(Dense(28, activation = 'sigmoid'))
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy'])

In [ ]:
model.fit(img, y, epochs = 1, validation_split = 0.1, batch_size = 32)

In [ ]:
score = 0
for img_no in range(len(image_array)):
    test_y = image_array[img_no]
    ty = np.reshape(test_y, (1,268,182,3))
    pred_genre = model.predict(ty, verbose = 0)
    best_pred_genre = genres[np.argmax(pred_genre)]
    correct_genres = id_genre_dict[id_list[img_no]]
    if best_pred_genre in correct_genres:
        score +=1
print('accuracy = ', (score/len(image_array))*100, '%')

In [ ]:
img_no = np.random.randint(low = 0, high = 997)
print(title_dict[id_list[img_no]])
test_y = image_array[img_no]
test_y.shape
ty = np.reshape(test_y, (1,268,182,3))
pred_genre = model.predict(ty)
print("Predicted: \t",genres[np.argmax(pred_genre)])
print("Correct answers: ",*id_genre_dict[id_list[img_no]])
Image.fromarray(test_y)